## 04. UCに登録されたモデルをモデルサービングエンドポイントにデプロイ

02でUCに登録したモデルを、リモートから利用できるようにモデルサービングエンドポイントにデプロイします。

DBR 15.4 ML 以降をお使いください

<img src='https://github.com/komae5519pv/komae_dbdemos/blob/main/e2e_ML_20250629/_data/_imgs/4_model_endpoint.png?raw=true' width='1200' />

In [0]:
%run ./00_setup

### 1. モデルサービングエンドポイントにデプロイ
MLflowデプロイSDKを使用して、モデルサービングエンドポイントにUCに登録されたモデルをデプロイします

In [0]:
import mlflow
from mlflow.deployments import get_deploy_client
from mlflow.exceptions import MlflowException
from requests.exceptions import HTTPError

# MLflowクライアントの作成
mlflow_client = mlflow.MlflowClient()

deploy_client = get_deploy_client("databricks")
model_version = mlflow_client.get_model_version_by_alias(f"{MY_CATALOG}.{MY_SCHEMA}.{MODEL_NAME}", "prod")

endpoint_name = f"komae_{MODEL_NAME}"[:63]  # 63文字以内にサニタイズ

# エンドポイント設定（served_entitiesでエイリアス指定）
config = {
    "served_entities": [{
        "entity_name": f"{MY_CATALOG}.{MY_SCHEMA}.{MODEL_NAME}",
        "entity_version": model_version.version,
        "workload_size": "Small"
    }]
}

# deploy_client.create_endpoint(name=endpoint_name, config=config)
# エラーハンドリング付（新規デプロイ or 既存あればデプロイなし）
try:
    existing_endpoint = deploy_client.get_endpoint(endpoint_name)
    print(f"既存のエンドポイント '{endpoint_name}' が見つかりました。更新は行いません。")
except HTTPError as e:
    if e.response.status_code == 404:
        endpoint = deploy_client.create_endpoint(name=endpoint_name, config=config)
        print(f"エンドポイント '{endpoint_name}' を新規作成しました。")
    else:
        raise
except MlflowException as e:
    print(f"エラーが発生しました: {e}")
    raise

モデルサービングエンドポイントが「READY」状態になるまでノートブックで監視します。<br>
MLflow Deployments SDK（またはREST API）を使ってエンドポイント状態をポーリングします。

In [0]:
import time

def wait_for_endpoint_ready(client, endpoint_name, timeout=1000, check_interval=10):
    """
    エンドポイントがREADY状態になるまで監視する
    :param deploy_client: MLflow Deployments client
    :param endpoint_name: 監視対象のエンドポイント名
    :param timeout: 最大待機時間（秒）
    :param check_interval: 状態チェック間隔（秒）
    :return: True if ready, False if timeout
    """
    start_time = time.time()
    while True:
        endpoint = deploy_client.get_endpoint(endpoint_name)
        state_ready = endpoint.get("state", {}).get("ready", "NOT_READY")
        print(f"Endpoint state: {state_ready}")
        if state_ready == "READY":
            print("Endpoint is ready to receive traffic.")
            return True
        if time.time() - start_time > timeout:
            print("Timeout waiting for endpoint to be ready.")
            return False
        time.sleep(check_interval)

# 監視実行
wait_for_endpoint_ready(deploy_client, endpoint_name)
